In [ ]:
import pickle5 as pickle
with open('../../data/COVID-QA/top_N_ents_spacy-COVID_QA.pkl', 'rb') as f:
    top_N_ents = pickle.load(f)

In [ ]:
import wikipedia
from tqdm.auto import tqdm

search_res = {}

for i, ent in tqdm(enumerate(top_N_ents)):
    #Skipping those entities which don't return anything
    if wikipedia.search(ent) != []:
        search_res[ent] = wikipedia.search(str(ent), results=1)[0]

In [ ]:
import wikipedia
from transformers import AutoTokenizer, AutoModel
import torch

context_dict = {}

filtering = False
if filtering == True:
    filtering_threshold = 0.5
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    #We're consider scibert because pubmedbert assigns very high similarity for both related/unrelated terms.
    checkpoint = 'allenai/scibert_scivocab_uncased'
    
    model = AutoModel.from_pretrained(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    cos = torch.nn.CosineSimilarity(dim=0)
    model.to(device)

for ent,res in tqdm(search_res.items()):
    if filtering == False:
        try:
            context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
        except:
            continue
    else:
        encoded_input = tokenizer([ent, res], return_tensors='pt', padding=True)
        with torch.no_grad():
            output = model(**encoded_input)
        
        similarity = cos(output.pooler_output[0], output.pooler_output[1])
        '''
        we're taking less than here since the similarity scores for related terms seem to be lower than 
        unrelated ones.
        '''
        if similarity.item() < filtering_thresold:
            try:
                context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
            except:
                continue

In [ ]:
import pandas as pd
pd.DataFrame(context_dict.items(), columns = ['ent', 'text']).to_parquet('wiki_corpus.parquet', index=False)

In [ ]:
from pymed import PubMed

# Create a PubMed object that GraphQL can use to query
# Note that the parameters are not required but kindly requested by PubMed Central
# https://www.ncbi.nlm.nih.gov/pmc/tools/developers/
pubmed = PubMed(tool="MyTool", email="my@email.address")

# Create a GraphQL query in plain text
query = ['covid-19', 'hiv-1']

# Execute the query against the API
results = pubmed.query(query, max_results=500)

# Loop over the retrieved articles
for article in results:

    # Extract and format information from the article
    article_id = article.pubmed_id
    title = article.title
    abstract = article.abstract

    # Show information about the article
    print(
        f'{abstract}\n'
    )

In [2]:
from datasets import load_dataset
s = load_dataset('json', data_files='../../data/RadQA'\
                 '/radqa-a-question-answering-dataset-to-improve-comprehension-of-radiology-reports-1.0.0'\
                 '/train.json', field='data')

Found cached dataset json (/home/saptarshi/.cache/huggingface/datasets/json/default-abc7806a378e397d/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
all_contexts = []
all_q = []
for row in s['train']['paragraphs']:
    for d in row:
        all_contexts.append(d["context"])
        for q in d['qas']:
            all_q.append(q["question"])

In [41]:
import re
c=0
for r in all_contexts:
    #if re.search('findings',r, re.IGNORECASE) and re.search('impression:', r, re.IGNORECASE):
    #if re.search('final report', r, re.IGNORECASE) and re.search('impression:', r, re.IGNORECASE):
    if re.search('findings and impression', r, re.IGNORECASE):
        print(r)
        print('-'*100)
        

FINDINGS and IMPRESSION:

 Material contrast is demonstrated within the thecal sac.

 There is moderate spinal canal stenosis at L3-L4 and L4-L5 levels.

 There is no evidence of obstruction to the flow of contrast. Please refer to
 the dedicated CT myelogram of the lumbar spine for details.

 Please see the report of the CT of the lumbar spine for additional
 information.
----------------------------------------------------------------------------------------------------
FINDINGS AND IMPRESSION:  Perivascular calcifications identified throughout
 the foot. There is no evidence for acute fractures/dislocations.  Within the
 medial aspect of the medial cuneiform, near the first tarsometatarsal joint,
 there is a focal area of lucency which may be projectional.  No definite
 cortical breakthrough or periosteal reaction is identified.  However, there is
 no air within the adjacent soft tissues.  Would correlate with pain within
 this region.  The rest of the bony structures demonstrate no

In [41]:
a = 0
for row in s['train']['paragraphs']:
    for c in row:
        if c['context'] == 'IMPRESSION':
            print(c)
print(a)

{'context': 'IMPRESSION', 'document_id': '1136073_I', 'qas': [{'answers': [], 'id': '1136073_2_1_I', 'is_impossible': True, 'question': 'Is an abdominal obstruction present?'}, {'answers': [], 'id': '1136073_1_2_I', 'is_impossible': True, 'question': "Does the patient's abdomen show any signs of ovarian cancer metastatis?"}, {'answers': [], 'id': '1136073_1_1_I', 'is_impossible': True, 'question': 'Does the patient have any tissue/contents obstructing her bowels?'}]}
{'context': 'IMPRESSION', 'document_id': '1191659_I', 'qas': [{'answers': [], 'id': '1191659_2_1_I', 'is_impossible': True, 'question': 'How has GBS changed over the interval?'}, {'answers': [], 'id': '1191659_1_1_I', 'is_impossible': True, 'question': 'Did the chest X ray show any changes in the interim?'}]}
{'context': 'IMPRESSION', 'document_id': '779328_I', 'qas': [{'answers': [], 'id': '779328_2_1_I', 'is_impossible': True, 'question': 'What is the position of the ETT?'}, {'answers': [], 'id': '779328_1_1_I', 'is_impo